In [1]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_222"; OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~16.04.1-b10); OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
  Starting server from /home/gabe/Envs/h2o/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphocMYs
  JVM stdout: /tmp/tmphocMYs/h2o_gabe_started_from_python.out
  JVM stderr: /tmp/tmphocMYs/h2o_gabe_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Argentina/Buenos_Aires
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,1 month and 18 days
H2O cluster name:,H2O_from_python_gabe_iw2jmf
H2O cluster total nodes:,1
H2O cluster free memory:,3.431 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [2]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("./1audience_train.csv")
test = h2o.import_file("./1audience_test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# Identify predictors and response
x = train.columns
y = "er_label"
x.remove(y)
x.remove('C1')
x.remove('device_id')
x.remove('key_evs_count')

In [6]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [7]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [10]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20191120_113332,0.741396,0.184752,0.354778,0.214348,0.0459451
XGBoost_grid_1_AutoML_20191120_113332_model_1,0.739121,0.180512,0.367376,0.21275,0.0452625
XGBoost_2_AutoML_20191120_113332,0.737509,0.180512,0.350039,0.212753,0.0452639
XGBoost_grid_1_AutoML_20191120_113332_model_3,0.73741,0.180482,0.349933,0.21272,0.0452497
XGBoost_1_AutoML_20191120_113332,0.737196,0.180549,0.367831,0.212789,0.0452794
StackedEnsemble_BestOfFamily_AutoML_20191120_113332,0.737192,0.185049,0.367367,0.214437,0.0459831
XGBoost_grid_1_AutoML_20191120_113332_model_4,0.736889,0.180558,0.367982,0.212796,0.0452821
XRT_1_AutoML_20191120_113332,0.736579,0.181159,0.349737,0.213003,0.0453702
GBM_5_AutoML_20191120_113332,0.736371,0.180504,0.349918,0.212749,0.0452621
XGBoost_3_AutoML_20191120_113332,0.736198,0.180499,0.367469,0.21276,0.045267


In [11]:
# The leader model is stored here
# aml.leader

In [12]:
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly
preds = aml.leader.predict(test)
aml.leader

stackedensemble prediction progress: |████████████████████████████████████| 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20191120_113332

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0456527210307
RMSE: 0.213664973804
LogLoss: 0.183816597036
Null degrees of freedom: 105998
Residual degrees of freedom: 105995
Null deviance: 44277.6126755
Residual deviance: 38968.7509384
AIC: 38976.7509384
AUC: 0.740683935547
pr_auc: 0.263562846203
Gini: 0.481367871094

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.104327053703: 


,0,1,Error,Rate
0,95309.0,5013.0,0.05,(5013.0/100322.0)
1,3878.0,1799.0,0.6831,(3878.0/5677.0)
Total,99187.0,6812.0,0.0839,(8891.0/105999.0)



Maximum Metrics: Maximum metrics at their respective thresholds


metric,threshold,value,idx
max f1,0.1043271,0.2880935,246.0
max f2,0.0467830,0.3634815,311.0
max f0point5,0.2332020,0.3516619,181.0
max accuracy,0.6494316,0.9491788,113.0
max precision,0.9820877,1.0,0.0
max recall,0.0315190,1.0,364.0
max specificity,0.9820877,1.0,0.0
max absolute_mcc,0.2332020,0.2624047,181.0
max min_per_class_accuracy,0.0446314,0.6297024,320.0
max mean_per_class_accuracy,0.0448459,0.6906400,319.0



Gains/Lift Table: Avg response rate:  5.36 %, avg score:  5.44 %


...

,0,1,Error,Rate
0,93993.0,6329.0,0.0631,(6329.0/100322.0)
1,3670.0,2007.0,0.6465,(3670.0/5677.0)
Total,97663.0,8336.0,0.0943,(9999.0/105999.0)



Maximum Metrics: Maximum metrics at their respective thresholds


metric,threshold,value,idx
max f1,0.0868086,0.2864483,305.0
max f2,0.0484741,0.3627993,350.0
max f0point5,0.2095006,0.3411718,213.0
max accuracy,0.6945769,0.9491222,85.0
max precision,0.9861215,1.0,0.0
max recall,0.0312401,1.0,399.0
max specificity,0.9861215,1.0,0.0
max absolute_mcc,0.2095006,0.2536132,213.0
max min_per_class_accuracy,0.0421791,0.6762325,365.0
max mean_per_class_accuracy,0.0466803,0.6903303,351.0



Gains/Lift Table: Avg response rate:  5.36 %, avg score:  5.36 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100756,0.4730570,11.2414568,11.2414568,0.6020599,0.7493076,0.6020599,0.7493076,0.1132640,0.1132640,1024.1456755,1024.1456755
,2,0.0201228,0.1593493,5.2420898,8.2459922,0.2807512,0.2575787,0.4416315,0.5037889,0.0526687,0.1659327,424.2089777,724.5992230
,3,0.0300003,0.1174456,4.0838678,6.8756324,0.2187202,0.1368101,0.3682390,0.3829629,0.0403382,0.2062709,308.3867796,587.5632393
,4,0.0401230,0.1105805,3.1670472,5.9399842,0.1696179,0.1144121,0.3181284,0.3152096,0.0320592,0.2383301,216.7047229,493.9984173
,5,0.0501420,0.1066153,3.3405037,5.4205772,0.1789077,0.1087074,0.2903104,0.2739480,0.0334684,0.2717985,234.0503708,442.0577163
,6,0.1012085,0.0697981,2.5456647,3.9699898,0.1363384,0.0873454,0.2126212,0.1797944,0.1299982,0.4017967,154.5664748,296.9989831
,7,0.1510203,0.0565536,1.7646131,3.2425792,0.0945076,0.0620013,0.1736632,0.1409421,0.0878985,0.4896953,76.4613092,224.2579212
,8,0.2032189,0.0466780,1.4139571,2.7728811,0.0757275,0.0525793,0.1485075,0.1182453,0.0738066,0.5635018,41.3957079,177.2881136
,9,0.3041915,0.0433164,0.8949416,2.1495222,0.0479305,0.0445379,0.1151222,0.0937790,0.0903646,0.6538665,-10.5058364,114.9522171
,10,0.4143813,0.0378378,0.8312724,1.7989813,0.0445205,0.0409826,0.0963482,0.0797397,0.0915977,0.7454642,-16.8727574,79.8981304
